# Заготовка

# ДЗ 6 Гладышев В.В.

In [1]:
# %pip install seaborn

In [238]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import seaborn as sns
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report


from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from scipy.stats import ttest_rel

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score, cross_validate

from typing import List, Optional, Dict, Tuple

data_dir = "geekbrains-competitive-data-analysis/"

! is disabled. For pip use '%pip ...'


In [233]:
import lightgbm as lgbm

In [ ]:
# !pip install catboost

In [234]:
import catboost as catb

## Загрузка данных

In [ ]:
%pip install dulwich

import dulwich.client
from dulwich.repo import Repo
from dulwich import index

import os
import shutil

def git_clone(src, target):
    client, path = dulwich.client.get_transport_and_path(src)
    if os.path.isdir(target):
        shutil.rmtree(target)
    os.makedirs(target)
    r = Repo.init(target)

    remote_refs = client.fetch(src, r)
    r[b"HEAD"] = remote_refs.refs[b"HEAD"]

    index.build_index_from_tree(r.path, r.index_path(), r.object_store, r[b'HEAD'].tree)

src = "https://github.com/VitalyGladyshev/Python_for_DS" # "https://github.com/VitalyGladyshev/Python_for_DS" #  # https://github.com/githubtraining/hellogitworld
target = "/home/jupyter/work/resources/pds"

git_clone(src, target)
os.listdir(target)

In [246]:
from cloud_ml.storage.api import Storage

# To retrieve client secret:
# 1. Go to link: https://developers.google.com/drive/api/v3/quickstart/python
# 2. Press "Enable the drive API"
# 3. Choose "TVs and limited input devices"
client_secret = {"installed":{"client_id":"50654617259-cf08c4m3451grc6a1lt3v5itin4oa7gn.apps.googleusercontent.com","project_id":"quickstart-1601663160688","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"08IiCKrHl77uNCacllOd54Wl","redirect_uris":["urn:ietf:wg:oauth:2.0:oob","http://localhost"]}}

# downloading contents of the remote file into the local one
gdrive = Storage.gdrive(client_secret)
gdrive_file_id = '10S2TyPJkOi4I5xWPLWTYyqZqt_NsyuqN'
dst_path = 'applications_history.csv'
gdrive.get(gdrive_file_id, dst_path)

In [248]:
from cloud_ml.storage.api import Storage

# To retrieve application id and secret:
# 1. Go to link: https://oauth.yandex.ru/client/new
# 2. Choose 'Web services'
# 3. Paste into 'Callback URI': https://oauth.yandex.ru/verification_code
# 4. Set up permissions on yandex disk
disk = Storage.ya_disk(application_id='dd6a450efee04064a6edd9fe65ad43ca', application_secret='91f47f36c99a457a923a0ac99bf23150')

# downloading contents of the remote file into the local one
# disk.get('path/to/file/within/ya/disk/file.txt', 'path/to/file.txt')
disk.get('Программирование/DS/GB/Факультативы/Спортивный анализ данных. Платформа Kaggle/geekbrains-competitive-data-analysis.zip', 'geekbrains-competitive-data-analysis.zip')
# https://yadi.sk/d/MQaV4BywVoPwAw

/py-env/platform-env/lib/python3.7/site-packages/ml_kernel/kernel.py:434: UserWarning: The following variables cannot be serialized: disk
  warnings.warn(message)


In [249]:
import zipfile
from tqdm import tqdm

fname = './geekbrains-competitive-data-analysis.zip'
path = './'

with zipfile.ZipFile(fname, 'r') as zf:
    for entry in tqdm(zf.infolist(), desc='Extracting '):
        try:
            zf.extract(entry, path)
        except zipfile.error as e:
            pass

Extracting : 100%|██████████| 7/7 [01:16<00:00, 10.88s/it]
/py-env/platform-env/lib/python3.7/site-packages/ml_kernel/kernel.py:434: UserWarning: The following variables cannot be serialized: zf
  warnings.warn(message)


In [250]:
src_list = ["train", "test", "client_profile", "applications_history", "bki", "payments"]

In [251]:
data_dir = './'

In [252]:
source = {}
for name in src_list:
    source[name] = pd.read_csv(data_dir + name + ".csv")

## Ключевые характеристики

In [253]:
pd.set_option('display.max_columns', 400)

In [254]:
for name in src_list:
    print(f"{name} \n {source[name].shape} \n")

train 
 (110093, 3) 

test 
 (165141, 2) 

client_profile 
 (250000, 24) 

applications_history 
 (1670214, 26) 

bki 
 (945234, 17) 

payments 
 (1023932, 8) 



In [255]:
source['train'].head(3)

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash
1,123597908,1,Cash
2,123526683,0,Cash


In [256]:
source['client_profile'].head(3)

,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,8560,1549,NaN,1,0,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,23187,365243,NaN,0,0,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0
2,123501780,M,1,427500.0,239850.0,23850.0,Incomplete higher,Married,0.072508,14387,326,18.0,0,0,3.0,0.409017,0.738159,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [263]:
source['test'].head(3)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE
0,123724268,Cash
1,123456549,Cash
2,123428178,Credit Card


In [264]:
for name in src_list:
    print(f"{name} \n")
    source[name].info()
    print("\n")

train 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110093 entries, 0 to 110092
Data columns (total 3 columns):
APPLICATION_NUMBER    110093 non-null int64
TARGET                110093 non-null int64
NAME_CONTRACT_TYPE    110093 non-null object
dtypes: int64(2), object(1)
memory usage: 2.5+ MB


test 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165141 entries, 0 to 165140
Data columns (total 2 columns):
APPLICATION_NUMBER    165141 non-null int64
NAME_CONTRACT_TYPE    165141 non-null object
dtypes: int64(1), object(1)
memory usage: 2.5+ MB


client_profile 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 24 columns):
APPLICATION_NUMBER            250000 non-null int64
GENDER                        250000 non-null object
CHILDRENS                     250000 non-null int64
TOTAL_SALARY                  250000 non-null float64
AMOUNT_CREDIT                 250000 non-null float64
AMOUNT_ANNUITY                249989 non-null 

In [265]:
def data_obj_discribe(data: pd.DataFrame):
# если столбец имеет тип 'object' и содержит менее 30 уникальных 
# распечатывает значения столбцов и подсчитывает количество значений

    # data: pd.DataFrame исходный датафрейм
    
    obj_fit = data.select_dtypes(include='object')
    print('*' * 100)
    for cat_colname in obj_fit.columns:
        if len(obj_fit[cat_colname].unique()) < 30:
            print(str(cat_colname) + '\n\n' + str(obj_fit[cat_colname].unique()) + '\n\n')
            print(obj_fit.groupby(cat_colname)[cat_colname].count())
            print('*' * 100 + '\n')

In [266]:
# for name in src_list:
#     obj_col = source[name].select_dtypes(include='object')
#     if not obj_col.empty:
#         print(f"\t{name}{source[name].shape}\n")
#         data_obj_discribe(source[name])

## Формируем "Большую таблицу"

### Подшиваем test к train

In [267]:
test = source['test']

In [268]:
test["TARGET"] = math.nan

In [269]:
test.head

<bound method NDFrame.head of         APPLICATION_NUMBER NAME_CONTRACT_TYPE  TARGET
0                123724268               Cash     NaN
1                123456549               Cash     NaN
2                123428178        Credit Card     NaN
3                123619984               Cash     NaN
4                123671104               Cash     NaN
...                    ...                ...     ...
165136           123487967               Cash     NaN
165137           123536402               Cash     NaN
165138           123718238               Cash     NaN
165139           123631557               Cash     NaN
165140           123433260               Cash     NaN

[165141 rows x 3 columns]>

In [270]:
data = pd.concat([source['train'], test], axis=0)
data.tail()

/py-env/platform-env/lib/python3.7/site-packages/ml_kernel/kernel.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import os


,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET
165136,123487967,Cash,NaN
165137,123536402,Cash,NaN
165138,123718238,Cash,NaN
165139,123631557,Cash,NaN
165140,123433260,Cash,NaN


In [271]:
data.shape

(275234, 3)

### Добавляем client_profile

In [272]:
def create_client_profile_features(X: pd.DataFrame, copy: bool = True) -> pd.DataFrame:
    """
    Создание признаков на основе профиля клиентов.

    Parameters
    ----------
    X: pandas.core.frame.DataFrame
        Матрица признаков с исходным профилем клиента.

    copy: bool, optional, default = True
        Флаг использования копии датафрейма X.
        Опциональный параметр, по умолчанию, равен True.

    Returns
    -------
    X_transformed: pandas.core.frame.DataFrame
        Расширенная матрица признаков с профилем клиентов.

    """
    if copy:
        X = X.copy()

    X["DAYS_ON_LAST_JOB"] = X["DAYS_ON_LAST_JOB"].replace(365243, np.nan)
    bki_flags = [flag for flag in X.columns if "AMT_REQ_CREDIT_BUREAU" in flag]
    X["BKI_REQUESTS_COUNT"] = X[bki_flags].sum(axis=1)
    X["BKI_KURTOSIS"] = X[bki_flags].kurtosis(axis=1)

    X["EXTERNAL_SCORING_PROD"] = X["EXTERNAL_SCORING_RATING_1"] * X["EXTERNAL_SCORING_RATING_2"] * X["EXTERNAL_SCORING_RATING_3"]
    X["EXTERNAL_SCORING_WEIGHTED"] = X.EXTERNAL_SCORING_RATING_1 * 2 + X.EXTERNAL_SCORING_RATING_2 * 1 + X.EXTERNAL_SCORING_RATING_3 * 3

    for function_name in ["min", "max", "mean", "nanmedian", "var"]:
        feature_name = "EXTERNAL_SCORING_RATING_{}".format(function_name)
        X[feature_name] = eval("np.{}".format(function_name))(
            X[["EXTERNAL_SCORING_RATING_1", "EXTERNAL_SCORING_RATING_2", "EXTERNAL_SCORING_RATING_3"]], axis=1
        )

    # Отношение между основными фин. показателями
    X['RATIO_CREDIT_TO_ANNUITY'] = X['AMOUNT_CREDIT'] / X['AMOUNT_ANNUITY']
    X["RATIO_ANNUITY_TO_SALARY"] = X['AMOUNT_ANNUITY'] / X['TOTAL_SALARY']
    X['RATIO_CREDIT_TO_SALARY'] = X['AMOUNT_CREDIT'] / X['TOTAL_SALARY']
    #X["TOTAL_SALARY_NET"] = X["TOTAL_SALARY"] - X["AMOUNT_ANNUITY"]

    # Отношение фин. показателей к возрасту и временным фичам
    X["RATIO_ANNUITY_TO_AGE"] = X["AMOUNT_ANNUITY"] / X["AGE"]
    X["RATIO_CREDIT_TO_AGE"] = X["AMOUNT_CREDIT"] / X["AGE"]
    X["RATIO_SALARY_TO_AGE"] = X["TOTAL_SALARY"] / X["AGE"]
    X["RATIO_SALARY_TO_EXPERIENCE"] = X["TOTAL_SALARY"] / X["DAYS_ON_LAST_JOB"]
    X["RATIO_CREDIT_TO_EXPERIENCE"] = X["AMOUNT_CREDIT"] / X["DAYS_ON_LAST_JOB"]
    X["RATIO_ANNUITY_TO_EXPERIENCE"] = X["AMOUNT_ANNUITY"] / X["DAYS_ON_LAST_JOB"]

    # Отношение врменных признаков
    X["RATIO_AGE_TO_EXPERIENCE"] = X["AGE"] / X["DAYS_ON_LAST_JOB"]
    X["RATIO_SALARY_TO_REGION_POPULATION"] = X["TOTAL_SALARY"] * X["REGION_POPULATION"]
    X["RATIO_CAR_TO_EXPERIENCE"] = X["OWN_CAR_AGE"] / X["DAYS_ON_LAST_JOB"]
    X["RATIO_CAR_TO_AGE"] = X["OWN_CAR_AGE"] / X["AGE"]

    # Произведение фин. показателей кредита на вероятность дефолта
    # Такая штука называется математическим ожиданием дефолта или ожидаемыми потерями
    X["EXPECTED_TOTAL_LOSS_1"] = X["EXTERNAL_SCORING_RATING_1"] * X["AMOUNT_CREDIT"]
    X["EXPECTED_TOTAL_LOSS_2"] = X["EXTERNAL_SCORING_RATING_2"] * X["AMOUNT_CREDIT"]
    X["EXPECTED_TOTAL_LOSS_3"] = X["EXTERNAL_SCORING_RATING_3"] * X["AMOUNT_CREDIT"]
    X["EXPECTED_MONTHLY_LOSS_1"] = X["EXTERNAL_SCORING_RATING_1"] * X["AMOUNT_ANNUITY"]
    X["EXPECTED_MONTHLY_LOSS_2"] = X["EXTERNAL_SCORING_RATING_2"] * X["AMOUNT_ANNUITY"]
    X["EXPECTED_MONTHLY_LOSS_3"] = X["EXTERNAL_SCORING_RATING_3"] * X["AMOUNT_ANNUITY"]

    return X

In [273]:
client_profile = create_client_profile_features(source["client_profile"])
client_profile.head(n=2)

/py-env/platform-env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


,APPLICATION_NUMBER,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3
0,123666076,F,0,157500.0,270000.0,13500.0,Incomplete higher,Civil marriage,0.008068,8560,1549.0,NaN,1,0,2.0,0.329471,0.236315,0.678568,0.0,0.0,0.0,0.0,1.0,2.0,3.0,1.428571,0.052832,2.930959,0.236315,0.678568,0.414784,0.329471,0.036237,20.00000,0.085714,1.714286,1.577103,31.542056,18.399533,101.678502,174.306004,8.7153,5.526146,1270.71,NaN,NaN,88957.124333,63804.966560,183213.275945,4447.856217,3190.248328,9160.663797
1,123423688,F,0,270000.0,536917.5,28467.0,Secondary / secondary special,Married,0.020246,23187,NaN,NaN,0,0,2.0,NaN,0.442295,0.802745,0.0,0.0,0.0,0.0,1.0,1.0,2.0,-1.875000,NaN,NaN,0.442295,0.802745,0.622520,0.622520,0.032481,18.86105,0.105433,1.988583,1.227714,23.155971,11.644456,NaN,NaN,NaN,NaN,5466.42,NaN,NaN,NaN,237475.743779,431008.094056,NaN,12590.802122,22851.755462


In [274]:
data = pd.merge(data, client_profile, on='APPLICATION_NUMBER', how='left')
data.head()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3
0,123687442,Cash,0.0,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,0.324353,4.197193,0.645914,0.716570,0.687756,0.700784,9.169231e-04,34.025788,0.159543,5.428571,1.597660,54.361648,10.013988,91.623037,497.382199,14.617801,9.149506,3008.4075,0.006399,0.000699,599170.547652,552256.266546,612667.559305,17609.307043,16230.521013,18005.977111
1,123597908,Cash,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123526683,Cash,0.0,F,0.0,135000.0,1006920.0,42660.0,Higher education,Married,0.026392,21557.0,3618.0,NaN,1.0,0.0,2.0,NaN,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0,11.0,0.539379,NaN,NaN,0.267869,0.682149,0.475009,0.475009,4.290708e-02,23.603376,0.316000,7.458667,1.978940,46.709653,6.262467,37.313433,278.308458,11.791045,5.958264,3562.9200,NaN,NaN,NaN,686869.876357,269722.588880,NaN,29100.493510,11427.288803
3,123710391,Cash,1.0,M,0.0,180000.0,518562.0,22972.5,Secondary / secondary special,Married,0.031329,22338.0,NaN,NaN,0.0,0.0,2.0,NaN,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,0.170446,0.171299,0.170873,0.170873,1.818800e-07,22.573164,0.127625,2.880900,1.028405,23.214343,8.058018,NaN,NaN,NaN,NaN,5639.2200,NaN,NaN,NaN,88829.188848,88386.882459,NaN,3935.167908,3915.573562
4,123590329,Cash,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [275]:
data.tail()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3
275229,123487967,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275230,123536402,Cash,NaN,M,0.0,135000.0,450000.0,16807.5,Secondary / secondary special,Single / not married,0.028663,18610.0,10398.0,23.0,0.0,0.0,1.0,NaN,0.665971,0.535276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,0.535276,0.665971,0.600624,0.600624,0.004270,26.773762,0.124500,3.333333,0.903143,24.180548,7.254164,12.983266,43.277553,1.616417,1.789767,3869.5050,0.002212,0.001236,NaN,299686.841680,240874.312713,NaN,11193.303537,8996.655580
275231,123718238,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275232,123631557,Cash,NaN,F,0.0,112500.0,350181.0,36769.5,Secondary / secondary special,Married,0.030755,20551.0,NaN,NaN,0.0,1.0,2.0,NaN,0.761376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.761376,0.761376,0.761376,0.761376,0.000000,9.523681,0.326840,3.112720,1.789183,17.039609,5.474186,NaN,NaN,NaN,NaN,3459.9375,NaN,NaN,NaN,266619.257702,NaN,NaN,27995.398940,NaN
275233,123433260,Cash,NaN,M,0.0,247500.0,1241437.5,36427.5,Secondary / secondary special,Married,0.018029,18853.0,6543.0,15.0,0.0,0.0,2.0,NaN,0.493617,0.546023,0.0,0.0,0.0,1.0,0.0,2.0,3.0,1.428571,NaN,NaN,0.493617,0.546023,0.519820,0.519820,0.000687,34.079679,0.147182,5.015909,1.932186,65.848273,13.127884,37.826685,189.735213,5.567400,2.881400,4462.1775,0.002293,0.000796,NaN,612794.525674,677853.672632,NaN,17981.229489,19890.260009


In [276]:
data.shape

(275234, 54)

### Заполняем пропуски

In [283]:
obj_col_names = data.select_dtypes(include=['object']).columns
obj_col_names

Index(['NAME_CONTRACT_TYPE', 'GENDER', 'EDUCATION_LEVEL', 'FAMILY_STATUS'], dtype='object')

In [284]:
data.loc[data[obj_col_names[1]].isnull(), obj_col_names[1]]

1         NaN
4         NaN
10        NaN
16        NaN
17        NaN
         ... 
275225    NaN
275226    NaN
275228    NaN
275229    NaN
275231    NaN
Name: GENDER, Length: 51519, dtype: object

In [285]:
for col in obj_col_names:
    data.loc[data[col].isnull(), col] = 'No'

In [286]:
num_col_names = data.select_dtypes(exclude=['object']).columns
num_col_names

Index(['APPLICATION_NUMBER', 'TARGET', 'CHILDRENS', 'TOTAL_SALARY',
       'AMOUNT_CREDIT', 'AMOUNT_ANNUITY', 'REGION_POPULATION', 'AGE',
       'DAYS_ON_LAST_JOB', 'OWN_CAR_AGE', 'FLAG_PHONE', 'FLAG_EMAIL',
       'FAMILY_SIZE', 'EXTERNAL_SCORING_RATING_1', 'EXTERNAL_SCORING_RATING_2',
       'EXTERNAL_SCORING_RATING_3', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'BKI_REQUESTS_COUNT', 'BKI_KURTOSIS',
       'EXTERNAL_SCORING_PROD', 'EXTERNAL_SCORING_WEIGHTED',
       'EXTERNAL_SCORING_RATING_min', 'EXTERNAL_SCORING_RATING_max',
       'EXTERNAL_SCORING_RATING_mean', 'EXTERNAL_SCORING_RATING_nanmedian',
       'EXTERNAL_SCORING_RATING_var', 'RATIO_CREDIT_TO_ANNUITY',
       'RATIO_ANNUITY_TO_SALARY', 'RATIO_CREDIT_TO_SALARY',
       'RATIO_ANNUITY_TO_AGE', 'RATIO_CREDIT_TO_AGE', 'RATIO_SALARY_TO_AGE',
       'RATIO_SALARY_TO_EXPERIEN

In [287]:
for col in num_col_names:
    data.loc[data[col].isnull(), col] = -1

In [288]:
data.head()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3
0,123687442,Cash,0.0,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,0.324353,4.197193,0.645914,0.716570,0.687756,0.700784,9.169231e-04,34.025788,0.159543,5.428571,1.597660,54.361648,10.013988,91.623037,497.382199,14.617801,9.149506,3008.4075,0.006399,0.000699,599170.547652,552256.266546,612667.559305,17609.307043,16230.521013,18005.977111
1,123597908,Cash,1.0,No,-1.0,-1.0,-1.0,-1.0,No,No,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2,123526683,Cash,0.0,F,0.0,135000.0,1006920.0,42660.0,Higher education,Married,0.026392,21557.0,3618.0,-1.0,1.0,0.0,2.0,-1.000000,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0,11.0,0.539379,-1.000000,-1.000000,0.267869,0.682149,0.475009,0.475009,4.290708e-02,23.603376,0.316000,7.458667,1.978940,46.709653,6.262467,37.313433,278.308458,11.791045,5.958264,3562.9200,-1.000000,-1.000000,-1.000000,686869.876357,269722.588880,-1.000000,29100.493510,11427.288803
3,123710391,Cash,1.0,M,0.0,180000.0,518562.0,22972.5,Secondary / secondary special,Married,0.031329,22338.0,-1.0,-1.0,0.0,0.0,2.0,-1.000000,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.000000,-1.000000,0.170446,0.171299,0.170873,0.170873,1.818800e-07,22.573164,0.127625,2.880900,1.028405,23.214343,8.058018,-1.000000,-1.000000,-1.000000,-1.000000,5639.2200,-1.000000,-1.000000,-1.000000,88829.188848,88386.882459,-1.000000,3935.167908,3915.573562
4,123590329,Cash,1.0,No,-1.0,-1.0,-1.0,-1.0,No,No,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


### Преобразуем категориальные признаки в числовые

In [299]:
data_cat = data.copy()

In [300]:
obj_col = data.select_dtypes(exclude=[np.number])

In [301]:
if not obj_col.empty:
    data_obj_discribe(data)

****************************************************************************************************
NAME_CONTRACT_TYPE

['Cash' 'Credit Card']


NAME_CONTRACT_TYPE
Cash           248983
Credit Card     26251
Name: NAME_CONTRACT_TYPE, dtype: int64
****************************************************************************************************

GENDER

['M' 'No' 'F' 'XNA']


GENDER
F      147444
M       76267
No      51519
XNA         4
Name: GENDER, dtype: int64
****************************************************************************************************

EDUCATION_LEVEL

['Secondary / secondary special' 'No' 'Higher education' 'Lower secondary'
 'Incomplete higher' 'Academic degree']


EDUCATION_LEVEL
Academic degree                     120
Higher education                  54516
Incomplete higher                  7456
Lower secondary                    2736
No                                51519
Secondary / secondary special    158887
Name: EDUCATION_LEVEL, dtype: int64
*

In [302]:
def data_obj_map(data: pd.DataFrame):
# если столбец имеет тип 'object' и содержит менее 30 уникальных 
# преобразует категориальные признак в числовые

    # data: pd.DataFrame исходный датафрейм
    
    # изменяет датафрейм источник
    
    obj_fit = data.select_dtypes(include='object')
    for cat_colname in obj_fit.columns:
        map_tr = {}
        un_val = obj_fit[cat_colname].unique()
        if len(un_val) < 30:
            print(str(cat_colname) + '\n')
            for i in range(len(un_val)):
                map_tr[un_val[i]] = i
            print(map_tr)
            data[cat_colname] = data[cat_colname].map(map_tr)
            print('*' * 100 + '\n')

In [303]:
if not obj_col.empty:
    data_obj_map(data)

NAME_CONTRACT_TYPE

{'Cash': 0, 'Credit Card': 1}
****************************************************************************************************

GENDER

{'M': 0, 'No': 1, 'F': 2, 'XNA': 3}
****************************************************************************************************

EDUCATION_LEVEL

{'Secondary / secondary special': 0, 'No': 1, 'Higher education': 2, 'Lower secondary': 3, 'Incomplete higher': 4, 'Academic degree': 5}
****************************************************************************************************

FAMILY_STATUS

{'Married': 0, 'No': 1, 'Single / not married': 2, 'Separated': 3, 'Civil marriage': 4, 'Widow': 5, 'Unknown': 6}
****************************************************************************************************

EDUCATION_FAMILY_STATUS

{'Secondary / secondary special | Married': 0, 'No | No': 1, 'Higher education | Married': 2, 'Secondary / secondary special | Single / not married': 3, 'Higher education | Single / not marr

In [304]:
data.head()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
0,123687442,0,0.0,0,1.0,157500.0,855000.0,25128.0,0,0,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,0.324353,4.197193,0.645914,0.716570,0.687756,0.700784,9.169231e-04,34.025788,0.159543,5.428571,1.597660,54.361648,10.013988,91.623037,497.382199,14.617801,9.149506,3008.4075,0.006399,0.000699,599170.547652,552256.266546,612667.559305,17609.307043,16230.521013,18005.977111,0.184211,0.159543,52500.0,0,0
1,123597908,0,1.0,1,-1.0,-1.0,-1.0,-1.0,1,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.0,1,1
2,123526683,0,0.0,2,0.0,135000.0,1006920.0,42660.0,2,0,0.026392,21557.0,3618.0,-1.0,1.0,0.0,2.0,-1.000000,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0,11.0,0.539379,-1.000000,-1.000000,0.267869,0.682149,0.475009,0.475009,4.290708e-02,23.603376,0.316000,7.458667,1.978940,46.709653,6.262467,37.313433,278.308458,11.791045,5.958264,3562.9200,-1.000000,-1.000000,-1.000000,686869.876357,269722.588880,-1.000000,29100.493510,11427.288803,0.134072,0.316000,67500.0,0,2
3,123710391,0,1.0,0,0.0,180000.0,518562.0,22972.5,0,0,0.031329,22338.0,-1.0,-1.0,0.0,0.0,2.0,-1.000000,0.171299,0.170446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.000000,-1.000000,0.170446,0.171299,0.170873,0.170873,1.818800e-07,22.573164,0.127625,2.880900,1.028405,23.214343,8.058018,-1.000000,-1.000000,-1.000000,-1.000000,5639.2200,-1.000000,-1.000000,-1.000000,88829.188848,88386.882459,-1.000000,3935.167908,3915.573562,0.347114,0.127625,90000.0,0,0
4,123590329,0,1.0,1,-1.0,-1.0,-1.0,-1.0,1,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.0,1,1


In [305]:
data_cat.head(2)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
0,123687442,Cash,0.0,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.71657,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,0.324353,4.197193,0.645914,0.71657,0.687756,0.700784,0.000917,34.025788,0.159543,5.428571,1.59766,54.361648,10.013988,91.623037,497.382199,14.617801,9.149506,3008.4075,0.006399,0.000699,599170.547652,552256.266546,612667.559305,17609.307043,16230.521013,18005.977111,0.184211,0.159543,52500.0,0,Secondary / secondary special | Married
1,123597908,Cash,1.0,No,-1.0,-1.0,-1.0,-1.0,No,No,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.00000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.0,1,No | No


### Разделяем train и test

In [306]:
train_prep = data.loc[data['TARGET'] != -1]
train_prep.shape

(110093, 59)

In [307]:
train_cat_prep = data_cat.loc[data['TARGET'] != -1]
train_cat_prep.shape

(110093, 59)

In [308]:
train_prep.head(3)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
0,123687442,0,0.0,0,1.0,157500.0,855000.0,25128.0,0,0,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,0.324353,4.197193,0.645914,0.716570,0.687756,0.700784,0.000917,34.025788,0.159543,5.428571,1.59766,54.361648,10.013988,91.623037,497.382199,14.617801,9.149506,3008.4075,0.006399,0.000699,599170.547652,552256.266546,612667.559305,17609.307043,16230.521013,18005.977111,0.184211,0.159543,52500.0,0,0
1,123597908,0,1.0,1,-1.0,-1.0,-1.0,-1.0,1,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.0,1,1
2,123526683,0,0.0,2,0.0,135000.0,1006920.0,42660.0,2,0,0.026392,21557.0,3618.0,-1.0,1.0,0.0,2.0,-1.000000,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0,11.0,0.539379,-1.000000,-1.000000,0.267869,0.682149,0.475009,0.475009,0.042907,23.603376,0.316000,7.458667,1.97894,46.709653,6.262467,37.313433,278.308458,11.791045,5.958264,3562.9200,-1.000000,-1.000000,-1.000000,686869.876357,269722.588880,-1.000000,29100.493510,11427.288803,0.134072,0.316000,67500.0,0,2


In [309]:
train_cat_prep.head(3)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
0,123687442,Cash,0.0,M,1.0,157500.0,855000.0,25128.0,Secondary / secondary special,Married,0.019101,15728.0,1719.0,11.0,0.0,0.0,3.0,0.700784,0.645914,0.716570,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,0.324353,4.197193,0.645914,0.716570,0.687756,0.700784,0.000917,34.025788,0.159543,5.428571,1.59766,54.361648,10.013988,91.623037,497.382199,14.617801,9.149506,3008.4075,0.006399,0.000699,599170.547652,552256.266546,612667.559305,17609.307043,16230.521013,18005.977111,0.184211,0.159543,52500.0,0,Secondary / secondary special | Married
1,123597908,Cash,1.0,No,-1.0,-1.0,-1.0,-1.0,No,No,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.0,1,No | No
2,123526683,Cash,0.0,F,0.0,135000.0,1006920.0,42660.0,Higher education,Married,0.026392,21557.0,3618.0,-1.0,1.0,0.0,2.0,-1.000000,0.682149,0.267869,0.0,0.0,0.0,7.0,0.0,4.0,11.0,0.539379,-1.000000,-1.000000,0.267869,0.682149,0.475009,0.475009,0.042907,23.603376,0.316000,7.458667,1.97894,46.709653,6.262467,37.313433,278.308458,11.791045,5.958264,3562.9200,-1.000000,-1.000000,-1.000000,686869.876357,269722.588880,-1.000000,29100.493510,11427.288803,0.134072,0.316000,67500.0,0,Higher education | Married


In [310]:
source['train'].head()

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
0,123687442,0,Cash
1,123597908,1,Cash
2,123526683,0,Cash
3,123710391,1,Cash
4,123590329,1,Cash


In [311]:
train_prep.tail()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
110088,123458312,0,0.0,1,-1.0,-1.0,-1.0,-1.0,1,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.0,1,1
110089,123672463,0,0.0,2,0.0,175500.0,269550.0,12618.0,0,0,0.010032,10519.0,1354.0,-1.0,0.0,0.0,2.0,0.528096,0.471774,0.581484,0.0,0.0,0.0,0.0,0.0,2.0,2.0,6.000,0.144872,3.272417,0.471774,0.581484,0.527118,0.528096,0.002007,21.362340,0.071897,1.535897,1.199544,25.625059,16.684095,129.615953,199.076809,9.319055,7.768833,1760.6160,-1.000000,-1.000000,142348.303154,127166.558499,156738.932900,6663.516562,5952.838565,7337.161400,0.651085,0.071897,87750.0,0,0
110090,123723001,0,0.0,1,-1.0,-1.0,-1.0,-1.0,1,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.000000,1.0,1,1
110091,123554358,0,0.0,2,2.0,270000.0,1024740.0,49428.0,4,0,0.019101,15008.0,361.0,22.0,0.0,0.0,4.0,0.653826,0.651227,0.558507,0.0,0.0,0.0,1.0,0.0,1.0,2.0,-1.875,0.237806,3.634399,0.558507,0.653826,0.621186,0.651227,0.001966,20.731974,0.183067,3.795333,3.293443,68.279584,17.990405,747.922438,2838.614958,136.919668,41.573407,5157.2700,0.060942,0.001466,670001.539719,667338.205604,572324.081646,32317.305956,32188.840903,27605.865593,0.263481,0.183067,67500.0,0,10
110092,123676265,1,0.0,2,1.0,112500.0,337500.0,16875.0,0,0,0.031329,15781.0,258.0,-1.0,0.0,0.0,3.0,-1.000000,0.552043,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.000,-1.000000,-1.000000,0.552043,0.552043,0.552043,0.552043,0.000000,20.000000,0.150000,3.000000,1.069324,21.386477,7.128826,436.046512,1308.139535,65.406977,61.166667,3524.5125,-1.000000,-1.000000,-1.000000,186314.656055,-1.000000,-1.000000,9315.732803,-1.000000,0.333333,0.150000,37500.0,0,0


In [312]:
source['train'].tail()

,APPLICATION_NUMBER,TARGET,NAME_CONTRACT_TYPE
110088,123458312,0,Cash
110089,123672463,0,Cash
110090,123723001,0,Cash
110091,123554358,0,Cash
110092,123676265,0,Credit Card


In [313]:
test_prep = data.loc[data['TARGET'] == -1].copy().reset_index(drop=True)
test_prep.drop('TARGET', axis=1, inplace=True)
test_prep.shape

(165141, 58)

In [314]:
test_cat_prep = data_cat.loc[data_cat['TARGET'] == -1].copy().reset_index(drop=True)
test_cat_prep.drop('TARGET', axis=1, inplace=True)
test_cat_prep.shape

(165141, 58)

In [315]:
test_prep.head(3)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
0,123724268,0,0,0.0,117000.0,1125000.0,32895.0,0,0,0.028663,16007.0,2646.0,20.0,0.0,0.0,2.0,-1.0,0.628266,-1.000000,0.0,0.0,0.0,0.0,1.0,4.0,5.0,4.639906,-1.0,-1.0,0.628266,0.628266,0.628266,0.628266,0.000000,34.199726,0.281154,9.615385,2.055038,70.281752,7.309302,44.217687,425.170068,12.431973,6.049509,3353.5710,0.007559,0.001249,-1.0,706799.427892,-1.000000,-1.0,20666.815272,-1.000000,0.104000,0.281154,58500.0,0,0
1,123456549,0,2,2.0,81000.0,312768.0,17095.5,0,0,0.019689,10315.0,459.0,-1.0,0.0,0.0,4.0,-1.0,0.578161,0.188490,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,-1.0,-1.0,0.188490,0.578161,0.383325,0.383325,0.037961,18.295341,0.211056,3.861333,1.657344,30.321667,7.852642,176.470588,681.411765,37.245098,22.472767,1594.8090,-1.000000,-1.000000,-1.0,180830.265914,58953.494506,-1.0,9883.951718,3222.322825,0.258978,0.211056,20250.0,0,0
2,123428178,1,2,2.0,157500.0,450000.0,22500.0,0,0,0.019101,13016.0,977.0,-1.0,1.0,0.0,4.0,-1.0,0.152544,0.382502,0.0,0.0,0.0,0.0,1.0,6.0,7.0,5.393765,-1.0,-1.0,0.152544,0.382502,0.267523,0.267523,0.013220,20.000000,0.142857,2.857143,1.728642,34.572833,12.100492,161.207779,460.593654,23.029683,13.322416,3008.4075,-1.000000,-1.000000,-1.0,68644.984320,172125.811865,-1.0,3432.249216,8606.290593,0.350000,0.142857,39375.0,0,0


In [316]:
test_cat_prep.head(3)

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
0,123724268,Cash,M,0.0,117000.0,1125000.0,32895.0,Secondary / secondary special,Married,0.028663,16007.0,2646.0,20.0,0.0,0.0,2.0,-1.0,0.628266,-1.000000,0.0,0.0,0.0,0.0,1.0,4.0,5.0,4.639906,-1.0,-1.0,0.628266,0.628266,0.628266,0.628266,0.000000,34.199726,0.281154,9.615385,2.055038,70.281752,7.309302,44.217687,425.170068,12.431973,6.049509,3353.5710,0.007559,0.001249,-1.0,706799.427892,-1.000000,-1.0,20666.815272,-1.000000,0.104000,0.281154,58500.0,0,Secondary / secondary special | Married
1,123456549,Cash,F,2.0,81000.0,312768.0,17095.5,Secondary / secondary special,Married,0.019689,10315.0,459.0,-1.0,0.0,0.0,4.0,-1.0,0.578161,0.188490,0.0,0.0,1.0,0.0,0.0,2.0,3.0,1.428571,-1.0,-1.0,0.188490,0.578161,0.383325,0.383325,0.037961,18.295341,0.211056,3.861333,1.657344,30.321667,7.852642,176.470588,681.411765,37.245098,22.472767,1594.8090,-1.000000,-1.000000,-1.0,180830.265914,58953.494506,-1.0,9883.951718,3222.322825,0.258978,0.211056,20250.0,0,Secondary / secondary special | Married
2,123428178,Credit Card,F,2.0,157500.0,450000.0,22500.0,Secondary / secondary special,Married,0.019101,13016.0,977.0,-1.0,1.0,0.0,4.0,-1.0,0.152544,0.382502,0.0,0.0,0.0,0.0,1.0,6.0,7.0,5.393765,-1.0,-1.0,0.152544,0.382502,0.267523,0.267523,0.013220,20.000000,0.142857,2.857143,1.728642,34.572833,12.100492,161.207779,460.593654,23.029683,13.322416,3008.4075,-1.000000,-1.000000,-1.0,68644.984320,172125.811865,-1.0,3432.249216,8606.290593,0.350000,0.142857,39375.0,0,Secondary / secondary special | Married


In [317]:
source['test'].head()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET
0,123724268,Cash,NaN
1,123456549,Cash,NaN
2,123428178,Credit Card,NaN
3,123619984,Cash,NaN
4,123671104,Cash,NaN


In [318]:
test_prep.tail()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,GENDER,CHILDRENS,TOTAL_SALARY,AMOUNT_CREDIT,AMOUNT_ANNUITY,EDUCATION_LEVEL,FAMILY_STATUS,REGION_POPULATION,AGE,DAYS_ON_LAST_JOB,OWN_CAR_AGE,FLAG_PHONE,FLAG_EMAIL,FAMILY_SIZE,EXTERNAL_SCORING_RATING_1,EXTERNAL_SCORING_RATING_2,EXTERNAL_SCORING_RATING_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BKI_REQUESTS_COUNT,BKI_KURTOSIS,EXTERNAL_SCORING_PROD,EXTERNAL_SCORING_WEIGHTED,EXTERNAL_SCORING_RATING_min,EXTERNAL_SCORING_RATING_max,EXTERNAL_SCORING_RATING_mean,EXTERNAL_SCORING_RATING_nanmedian,EXTERNAL_SCORING_RATING_var,RATIO_CREDIT_TO_ANNUITY,RATIO_ANNUITY_TO_SALARY,RATIO_CREDIT_TO_SALARY,RATIO_ANNUITY_TO_AGE,RATIO_CREDIT_TO_AGE,RATIO_SALARY_TO_AGE,RATIO_SALARY_TO_EXPERIENCE,RATIO_CREDIT_TO_EXPERIENCE,RATIO_ANNUITY_TO_EXPERIENCE,RATIO_AGE_TO_EXPERIENCE,RATIO_SALARY_TO_REGION_POPULATION,RATIO_CAR_TO_EXPERIENCE,RATIO_CAR_TO_AGE,EXPECTED_TOTAL_LOSS_1,EXPECTED_TOTAL_LOSS_2,EXPECTED_TOTAL_LOSS_3,EXPECTED_MONTHLY_LOSS_1,EXPECTED_MONTHLY_LOSS_2,EXPECTED_MONTHLY_LOSS_3,RATIO_SALARY_TO_AMOUNT_CREDIT,RATIO_AMOUNT_ANNUITY_TO_SALARY,RATIO_SALARY_TO_PER_FAMILY_SIZE,FLG_MORE_THAN_30PERCENT_FOR_CREDIT,EDUCATION_FAMILY_STATUS
165136,123487967,0,1,-1.0,-1.0,-1.0,-1.0,1,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,1.000000,1.000000,1.0,1,1
165137,123536402,0,0,0.0,135000.0,450000.0,16807.5,0,2,0.028663,18610.0,10398.0,23.0,0.0,0.0,1.0,-1.0,0.665971,0.535276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.0,-1.0,0.535276,0.665971,0.600624,0.600624,0.004270,26.773762,0.124500,3.333333,0.903143,24.180548,7.254164,12.983266,43.277553,1.616417,1.789767,3869.5050,0.002212,0.001236,-1.0,299686.841680,240874.312713,-1.0,11193.303537,8996.655580,0.300000,0.124500,135000.0,0,3
165138,123718238,0,1,-1.0,-1.0,-1.0,-1.0,1,1,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,1.000000,1.000000,1.0,1,1
165139,123631557,0,2,0.0,112500.0,350181.0,36769.5,0,0,0.030755,20551.0,-1.0,-1.0,0.0,1.0,2.0,-1.0,0.761376,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.000000,-1.0,-1.0,0.761376,0.761376,0.761376,0.761376,0.000000,9.523681,0.326840,3.112720,1.789183,17.039609,5.474186,-1.000000,-1.000000,-1.000000,-1.000000,3459.9375,-1.000000,-1.000000,-1.0,266619.257702,-1.000000,-1.0,27995.398940,-1.000000,0.321262,0.326840,56250.0,0,0
165140,123433260,0,0,0.0,247500.0,1241437.5,36427.5,0,0,0.018029,18853.0,6543.0,15.0,0.0,0.0,2.0,-1.0,0.493617,0.546023,0.0,0.0,0.0,1.0,0.0,2.0,3.0,1.428571,-1.0,-1.0,0.493617,0.546023,0.519820,0.519820,0.000687,34.079679,0.147182,5.015909,1.932186,65.848273,13.127884,37.826685,189.735213,5.567400,2.881400,4462.1775,0.002293,0.000796,-1.0,612794.525674,677853.672632,-1.0,17981.229489,19890.260009,0.199366,0.147182,123750.0,0,0


In [319]:
source['test'].tail()

,APPLICATION_NUMBER,NAME_CONTRACT_TYPE,TARGET
165136,123487967,Cash,NaN
165137,123536402,Cash,NaN
165138,123718238,Cash,NaN
165139,123631557,Cash,NaN
165140,123433260,Cash,NaN


### Сохраняем train и test

In [320]:
train_prep.to_csv(data_dir + "train_prep.csv", sep=";", index=False)

In [321]:
test_prep.to_csv(data_dir + "test_prep.csv", sep=";", index=False)

In [322]:
train_cat_prep.to_csv(data_dir + "train_cat_prep.csv", sep=";", index=False)

In [323]:
test_cat_prep.to_csv(data_dir + "test_cat_prep.csv", sep=";", index=False)

In [324]:
# from google.colab import files
# files.upload()

In [325]:
# train_prep = pd.read_csv(data_dir + "train_prep.csv")

In [326]:
# test_prep = pd.read_csv(data_dir + "test_prep.csv")

## Валидация на LightGBM

In [327]:
def make_cross_validation_adapt_l(X: pd.DataFrame,
                                  y: pd.Series,
                                  params: Dict,
                                  metric: callable,
                                  cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    params: Dict
        Параметры модели.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0]) 
    
    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        model = lgbm.LGBMRegressor(num_leaves=params['num_leaves'],
                                #    max_depth=params['max_depth'], 
                                   n_estimators=params["n_estimators"],
                                   learning_rate=params['learning_rate'],
                                   subsample=0.5, 
                                   seed=42)

        model.fit(x_train, 
                  y_train, 
                  early_stopping_rounds=50, 
                  eval_set = [(x_valid, y_valid)], 
                  eval_metric = 'auc',
                  verbose=False)

        y_train_pred = model.predict(x_train)
        y_valid_pred = model.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(model)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [328]:
parameters_l = {
    'num_leaves': 31,
    'n_estimators': 100,
    'learning_rate': 0.1
}

In [329]:
cv_strategy = KFold(n_splits=5, random_state=42)

estimators_lgbm_f, oof_score_lgbm_f, fold_train_scores_lgbm_f, fold_valid_scores_lgbm_f, oof_predictions_lgbm_f = make_cross_validation_adapt_l(
    train_prep.drop(["APPLICATION_NUMBER", "TARGET"], axis=1),
    train_prep["TARGET"],
    params=parameters_l,
    metric=roc_auc_score, 
    cv_strategy=cv_strategy
)

/py-env/platform-env/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold: 1, train-observations = 88074, valid-observations = 22019
train-score = 0.7559, valid-score = 0.7188
Fold: 2, train-observations = 88074, valid-observations = 22019
train-score = 0.7754, valid-score = 0.7184
Fold: 3, train-observations = 88074, valid-observations = 22019
train-score = 0.7676, valid-score = 0.7285
Fold: 4, train-observations = 88075, valid-observations = 22018
train-score = 0.7608, valid-score = 0.724
Fold: 5, train-observations = 88075, valid-observations = 22018
train-score = 0.756, valid-score = 0.7253
CV-results train: 0.7631 +/- 0.007
CV-results valid: 0.723 +/- 0.004
OOF-score = 0.7221


In [330]:
lgbm_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    "learning_rate": 0.1,
    "num_leaves": 31,
    "random_state": 42 
    }

In [331]:
dftrainLGB = lgbm.Dataset(data = train_prep.drop(["APPLICATION_NUMBER", "TARGET"], axis=1), 
                          label = train_prep["TARGET"])
                        #   feature_name = list(x_train))

cv_results_l = lgbm.cv(
        lgbm_params,
        dftrainLGB,
        num_boost_round=1000,
        nfold=5,
        metrics='auc',
        early_stopping_rounds=10
        )

/py-env/platform-env/lib/python3.7/site-packages/ml_kernel/kernel.py:434: UserWarning: The following variables cannot be serialized: dftrainLGB
  warnings.warn(message)


In [332]:
print(f"lgbm.cv auc-mean: {cv_results_l['auc-mean'][len(cv_results_l['auc-mean'])-1]:.4f}, auc-stdv: {cv_results_l['auc-stdv'][len(cv_results_l['auc-stdv'])-1]:.4f}")

lgbm.cv auc-mean: 0.7214, auc-stdv: 0.0074
